### [IMPORTANT] This step need to be completed manually

A. Get the labels coordinates with any tool and place them inside `all_position.py` \
B. When the file is loaded these coordinates will be transform to YOLO format, \
C. Finally fill the `positions.py` file


#### 0 - Errase all folders  `data/obj` `label` `RESULT` `label_list`

In [1]:
from shutil import rmtree

import os
from os import pardir, remove
from os.path import join, abspath

current = os.getcwd()
folder = current.split("\\")[-1]
root = abspath(join(current, pardir))
result_path = f"{root}\\RESULT_IMAGES\\{folder}"
label = f"{current}\\label"
obj = f"{current}\\data\\obj"
label_list = f"{current}\\label_list.py"

try:
    rmtree(result_path, ignore_errors=True)
    rmtree(label, ignore_errors=True)
    rmtree(obj, ignore_errors=True)
    remove(label_list)
    print("deleted successfully")
except:
    pass

deleted successfully


### 1A. Create the `label_list.py`


In [2]:
import os
from os import pardir
from os.path import join, abspath

DEBUG = 0

current = os.getcwd()
folderName = current.split("\\")[-1]
root = abspath(join(current, pardir))

path = current + "\\image_groups\\"

dir = os.listdir(path)
folders: list[str] = []
for file in dir:
    filepath = f"{path}\\{file}"
    if os.path.isdir(filepath):
        folders.append(file)

with open("label_list.py", "w") as file:
    file.write("LABELS = [")
    for label in folders:
        file.write(f"'{label}', ")
    file.write("]")

print(path) if DEBUG else 0
print(folders) if DEBUG else 0

0

### 1B. Update the `obj.data` with the labels length and `obj.names`


In [3]:
from label_list import LABELS

original = ""
with open("data/obj.data", "r") as file:
    original = file.read()

with open("data/obj.data", "w") as file:
    lines = original.split("\n")
    lines[0] = f"classes = {len(folders)}"
    text = "\n".join(lines)
    file.writelines(text)

with open("./data/obj.names", "w") as file:
    text = "\n".join(LABELS)
    print(text)
    file.write("\n".join(LABELS))  # type: ignore

AWS_CALIB
AWS_FIELD
EXPOSURE_REQUIRED
MCU_CALIBRATION
MCU_CALIB_PASS
MCU_NO_MESSAGE
MU_BLOCKED
MU_EXPOSURE
MU_OFFLINE
MU_STANDBY
OK_GREY


### 1.C Create `label` folder


In [4]:
from positions import POSITIONS
from label_list import LABELS
import os

if not os.path.exists("label"):
    os.mkdir("label")

obj = {}

for folder in folders:
    path = f"{current}\\image_groups\\{folder}"
    dir = os.listdir(path)

    for file in dir:
        filepath = f"{path}\\{file}"
        if not os.path.isdir(filepath):
            try:
                obj[file].append(f"{folder}")  # type: ignore
            except KeyError:
                obj[file] = [f"{folder}"]  # type: ignore

array: list[str] = []
for filename, labels in obj.items():  # type: ignore
    filepath: str = f"{current}\\label\\{filename.replace('png', 'txt')}"  # type: ignore
    try:
        with open(filepath, "w") as file:
            for label in labels:  # type: ignore
                i = LABELS.index(label)  # type: ignore
                p = POSITIONS[label]["position"]
                text = f"{i} {p}"
                array.append(text)
                file.write(text + "\n")
    except:
        pass

print("\n".join(list(set(array))))

0.9383333333333334 0.9675 0.051666666666666666 0.03375
0.11916666666666667 0.985625 0.195 0.02125
0.5 0.4996875 0.33666666666666667 0.164375
0.25458333333333333 0.9096875 0.4875 0.071875
0.4533333333333333 0.93125 0.09 0.02875
4 0.12875 0.9609375 0.21416666666666667 0.026875
7 0.9383333333333334 0.9675 0.051666666666666666 0.03375
5 0.18208333333333335 0.9609375 0.32083333333333336 0.026875
2 0.5 0.4996875 0.33666666666666667 0.164375
10 0.4533333333333333 0.93125 0.09 0.02875
9 0.9383333333333334 0.9675 0.051666666666666666 0.03375
1 0.9 0.8203125 0.175 0.034375
0 0.9658333333333333 0.881875 0.041666666666666664 0.02875
3 0.18208333333333335 0.9609375 0.32083333333333336 0.026875
8 0.11916666666666667 0.985625 0.195 0.02125
6 0.9383333333333334 0.9675 0.051666666666666666 0.03375


### 1C. Create `data/obj` folder with .txt files


In [5]:
from shutil import copy

if not os.path.exists("data/obj"):
    os.mkdir("data/obj")

file_prefixes = [f.split(".")[0] for f in os.listdir("label") if f.endswith(".txt")]

for prefix in file_prefixes:
    copy(f"label/{prefix}.txt", f"data/obj/{prefix}.txt")
    copy(f"images/{prefix}.png", f"data/obj/{prefix}.png")

### 2. Create the files need to train the neural network


In [7]:
from label_list import LABELS
import glob

with open("./cfg/yolov4-tiny-custom_template.cfg", "r") as file:
    cfg_content = file.read()
    cfg_content = cfg_content.replace("_CLASS_NUMBER_", str(len(LABELS)))  # type: ignore
    cfg_content = cfg_content.replace(
        "_NUMBER_OF_FILTERS_", str((len(LABELS) + 5) * 3)  # type: ignore
    )
    cfg_content = cfg_content.replace(
        "_MAX_BATCHES_", str(max(6000, len(LABELS) * 2000))  # type: ignore
    )

with open("./cfg/yolov4-tiny-custom.cfg", "w") as file:
    file.write(cfg_content)


current_dir = "data/obj"

# Percentage of images to be used for the test set
percentage_test = 10

# Create and/or truncate train.txt and test.txt
file_train = open("data/train.txt", "w")
file_test = open("data/test.txt", "w")

# Populate train.txt and test.txt
counter = 1
index_test = round(100 / percentage_test)

for pathAndFilename in glob.iglob(os.path.join(current_dir, "*.png")):
    print(pathAndFilename)
    title, ext = os.path.splitext(os.path.basename(pathAndFilename))

    if counter == index_test:

        counter = 1

        file_test.write("data/obj" + "/" + title + ".png" + "\n")
    else:
        file_train.write("data/obj" + "/" + title + ".png" + "\n")
        counter = counter + 1

data/obj\a.png
data/obj\a1.png
data/obj\b.png
data/obj\b1.png
data/obj\c.png
data/obj\c1.png
data/obj\d.png
data/obj\d1.png
data/obj\e.png
data/obj\e1.png
data/obj\f.png
data/obj\f1.png
data/obj\g.png
data/obj\g1.png
data/obj\img_1.png
data/obj\img_10.png
data/obj\img_101.png
data/obj\img_102.png
data/obj\img_103.png
data/obj\img_104.png
data/obj\img_105.png
data/obj\img_109.png
data/obj\img_11.png
data/obj\img_110.png
data/obj\img_111.png
data/obj\img_14.png
data/obj\img_15.png
data/obj\img_16.png
data/obj\img_18.png
data/obj\img_2.png
data/obj\img_20.png
data/obj\img_23.png
data/obj\img_26.png
data/obj\img_28.png
data/obj\img_30.png
data/obj\img_31.png
data/obj\img_33.png
data/obj\img_35.png
data/obj\img_36.png
data/obj\img_38.png
data/obj\img_4.png
data/obj\img_40.png
data/obj\img_42.png
data/obj\img_45.png
data/obj\img_46.png
data/obj\img_47.png
data/obj\img_48.png
data/obj\img_49.png
data/obj\img_50.png
data/obj\img_51.png
data/obj\img_52.png
data/obj\img_53.png
data/obj\img_54.pn

### 3. Train the neural network


In [8]:
from subprocess import Popen, PIPE

current = os.getcwd()
root = abspath(join(current, pardir))

exe = abspath(join(root, join("darknet", "darknet.exe")))

data = "data/obj.data"
config = "cfg/yolov4-tiny-custom.cfg"
conv_37 = "conv/yolov11.aws.37"

process = Popen(
    [exe, "detector", "train", data, config, conv_37], stdout=PIPE, text=True
)
while True:
    output = process.stdout.readline().replace("\n", "")  # type: ignore
    if process.poll() is not None:
        break
    if output:
        print(output)
process.poll()

 CUDNN_HALF=1 
yolov4-tiny-custom
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
Create CUDA-stream - 0 
 Create cudnn-handle 0 
nms_kind: greedynms (1), beta = 0.600000 
nms_kind: greedynms (1), beta = 0.600000 
 seen 64, trained: 0 K-images (0 Kilo-batches_64) 
Weights are saved after: 10000 iterations. Last weights (*_last.weight) are stored every 100 iterations. 
Learning Rate: 0.00261, Momentum: 0.9, Decay: 0.0005
 Detection layer: 30 - type = 28 
 Detection layer: 37 - type = 28 
 If error occurs - run training with flag: -dont_show 
Loaded: 0.217000 seconds
1/22000: loss=0.1 hours left=-1.0
 1: 0.123497, 0.123497 avg loss, 0.000000 rate, 0.606000 seconds, 64 images, -1.000000 hours left
Loaded: 0.001000 seconds
2/22000: loss=0.1 hours left=5.0
 2: 0.112708, 0.122419 avg loss, 0.000000 rate, 0.494000 seconds, 128 images, 5.029216 hours left
Loaded: 0.001000 seconds
3/22000: loss=0.1 hours left=5.0
 3: 0.132478, 0.123424 avg loss, 0.000000 rate

### 4. Test the neural network


In [ ]:
%run test_label_jp.py

### 5 [SUGGESTED] Get pre-trained weights


In [ ]:
from subprocess import Popen, PIPE
import os
from os import pardir
from os.path import join, abspath

current = os.getcwd()
root = abspath(join(current, pardir))

exe = abspath(join(root, join("darknet", "darknet.exe")))

layers = 37
config = "cfg/yolov4-tiny-custom.cfg"
weight = "weight/aws_11icons.weights"
conv = f"conv/yolov11.aws.{layers}"

# ./darknet partial cfg/extraction.cfg extraction.weights extraction.conv.weights 24

process = Popen(
    [exe, "partial", config, weight, conv, f"{layers}"],
    stdout=PIPE,
    stderr=PIPE,
    shell=True,
    text=True,
)  # type
out, err = process.communicate()
print(out)
print(err)